In [146]:
import pandas as pd
import gpxpy
import os


# Práca

bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"


# Load the CSV data into a DataFrame

df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [147]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

In [148]:
df_bridges_all["Správca_úseku"].value_counts().keys()

Index(['SSC - IVSC ZA', 'SSC - IVSC KE', 'SSC - IVSC BB', 'BBRSC-JUH',
       'BBRSC-STRED', 'BBRSC-SEVER', 'SSC - IVSC BA', 'SC TN SK - TN',
       'SUC PSK - HE', 'SUC PSK - PO', 'SC TN SK - PD', 'SUC TTSK - TT',
       'SUC PSK - PP', 'SUC PSK - BJ', 'SC TN SK - PB', 'SC ŽSK - ORA',
       'SC ŽSK - HPOV', 'SC KSK - KS', 'SC ŽSK - LIP', 'SUC PSK - SK',
       'SC KSK - SNV', 'RSUC NR - LV', 'SUC PSK - SL', 'RSUC NR - NR',
       'SC ŽSK - KYS', 'SC KSK - MI', 'SC BSK', 'SC ŽSK - TUR',
       'SUC TTSK - SE', 'NDS - SSUR ZV', 'SC KSK - TV', 'RSUC NR - NZ', 'VPO',
       'NDS - SSUD PB', 'SUC PSK - VT', 'SC KSK - RV', 'SUC TTSK - DS',
       'NDS - SSUD LM', 'RSUC NR - TO', 'RSUC NR - KN', 'NDS - SSUR KE',
       'NDS - SSUD TN', 'NDS - SSUD BE', 'MAG-BA', 'NDS - SSUR CA',
       'NDS - SSUD BA', 'NDS - SSUR NB', 'NDS - SSUD PO', 'NDS - SSUR GA',
       'NDS - SSUD MENG', 'NDS - SSUD MA', 'NDS - SSUD TT', 'NDS - SSUD MT',
       'MAG-KE', 'TT', 'NEZN', 'PP-HO', 'CU', 'ZA-Invest', 'BR-

In [149]:
# Filter bridges in the Nitra region with Trieda_PK as "cesta I. triedy" or "cesta II. triedy"
filtered_bridges = df_bridges_all[

    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - TT') |
    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - SE') |
    (df_bridges_all["Správca_úseku"] == 'SUC TTSK - DS')
]

# Reindex the DataFrame
filtered_bridges.reset_index(drop=True, inplace=True)

filtered_bridges = filtered_bridges[filtered_bridges["n_2023"] != 0]

filtered_bridges = filtered_bridges[filtered_bridges["Material"]
                                    == "prefabrikovaný predpätý betón"]

filtered_bridges = filtered_bridges[
    (filtered_bridges["n_2024"] == 4) |
    (filtered_bridges["n_2024"] == 5)
]

# Replace double space with single space in each item of the "Predmet_premostenia" column
filtered_bridges["Predmet_premostenia"] = filtered_bridges["Predmet_premostenia"].str.replace(
    "  ", " ")


# Display the filtered DataFrame
filtered_bridges.shape

(63, 44)

In [150]:
import pandas as pd

path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Mosty.xlsx"

# Load the Excel file into a DataFrame
df = pd.read_excel(path)

# Display the first few rows to ensure it loaded correctly
print(df.head())

      ID                         GPS   Dlzka   PK  Intenz
0  M6425  48.586307824, 17.824516076   10.43  499   12447
1  M6684  48.628765366, 17.719734958   12.40  499    3931
2  M4553  48.588876768, 17.810173653   71.60  499   14134
3   M679  48.683035961, 17.255918415   17.10  500    6052
4  M4864  48.655586793, 17.042874752  111.56  500    3663


In [151]:
import os
import pandas as pd

# Get the list of files in the folder
folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes"
files = os.listdir(folder_path)


# Add columns to indicate if the files exist
df['clear_exists'] = df['ID'].apply(
    lambda bridge_id: f"{bridge_id}_clear.gpx" in files)
df['alt_exists'] = df['ID'].apply(
    lambda bridge_id: f"{bridge_id}_alt.gpx" in files)


In [152]:
# Function to extract route data for a given GPX file
def extract_route_data(gpx_file_path):
    with open(gpx_file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    route_data = []
    cumulative_distance = 0
    previous_point = None

    for track in gpx.tracks:
        for segment in track.segments:
            for index, point in enumerate(segment.points):
                if previous_point is not None:
                    distance = point.distance_2d(previous_point)
                    cumulative_distance += distance
                    time = (point.time - previous_point.time).total_seconds(
                    ) if point.time and previous_point.time else 0
                else:
                    distance = 0
                    time = 0

                route_data.append({
                    "point_index": index,
                    "longitude": point.longitude,
                    "latitude": point.latitude,
                    "distance": distance,
                    "time": time,
                    "elevation": point.elevation
                })

                previous_point = point

    return route_data

In [153]:
import gpxpy
import os
import pandas as pd

# Function to extract route data for a given GPX file


def extract_route_data(gpx_file_path):
    import gpxpy
    with open(gpx_file_path, 'r', encoding='utf-8') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    route_data = []
    cumulative_distance = 0
    previous_point = None

    for track in gpx.tracks:
        for segment in track.segments:
            for index, point in enumerate(segment.points):
                if previous_point is not None:
                    distance = point.distance_2d(previous_point)
                    cumulative_distance += distance
                else:
                    distance = 0

                route_data.append({
                    "point_index": index,
                    "longitude": point.longitude,
                    "latitude": point.latitude,
                    "distance": distance,  # Distance from the previous point
                    "cumulative_distance": cumulative_distance,  # Total distance up to this point
                    "elevation": point.elevation
                })

                previous_point = point

    return route_data

# Path to the folder with GPX files
folder_path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Routes"

# Load the Excel file into a DataFrame
path = r"C:\Users\relia\Documents\GitHub\Bridges\02_SC_TA\IoT\Mosty.xlsx"
df = pd.read_excel(path)

# Initialize new columns for route data
clear_routes = []
alt_routes = []

for _, row in df.iterrows():
    bridge_id = row["ID"]
    clear_file = os.path.join(folder_path, f"{bridge_id}_clear.gpx")
    alt_file = os.path.join(folder_path, f"{bridge_id}_alt.gpx")

    # Extract route data for the clear route
    if os.path.exists(clear_file):
        clear_route_data = extract_route_data(clear_file)
    else:
        clear_route_data = None
    clear_routes.append(clear_route_data)

    # Extract route data for the alt route
    if os.path.exists(alt_file):
        alt_route_data = extract_route_data(alt_file)
    else:
        alt_route_data = None
    alt_routes.append(alt_route_data)

# Add route data to the DataFrame
df["Clear_Route"] = clear_routes
df["Alt_Route"] = alt_routes

In [154]:
# Initialize lists to store cumulative distances for each bridge
clear_distances = []
alt_distances = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Clear Route cumulative distance
    if row["Clear_Route"] is not None:
        route = row["Clear_Route"]
        cumulative_distance = 0
        for point in route:
            cumulative_distance += point["distance"]
        # Convert to kilometers
        clear_distances.append(cumulative_distance / 1000)
    else:
        clear_distances.append(None)  # If Clear_Route is None, append None

    # Alt Route cumulative distance
    if row["Alt_Route"] is not None:
        route = row["Alt_Route"]
        cumulative_distance = 0
        for point in route:
            cumulative_distance += point["distance"]
        # Convert to kilometers
        alt_distances.append(cumulative_distance / 1000)
    else:
        alt_distances.append(None)  # If Alt_Route is None, append None

# Add the cumulative distances to the DataFrame
df["Clear_Distance"] = clear_distances
df["Alt_Distance"] = alt_distances

In [155]:
def extract_time(gpx_file):
    with open(gpx_file, "r", encoding="utf-8") as file:
        gpx_content = file.read()

    # Initialize total time
    total_time = 0.0

    # Extract <rtept> sections
    rtept_sections = gpx_content.split("<rtept")[1:]  # Split by <rtept>

    # Process each <rtept>
    for rtept in rtept_sections:
        if "<gh:time>" in rtept:
            start_time = rtept.find("<gh:time>") + len("<gh:time>")
            end_time = rtept.find("</gh:time>")
            time = float(rtept[start_time:end_time])
            total_time += time

    # Convert to minutes
    total_time_minutes = total_time / (1000 * 60)  # milliseconds to minutes
    return total_time_minutes


# Initialize lists to store total times for each route
clear_times = []
alt_times = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Clear Route total time
    if row["Clear_Route"] is not None:
        clear_file = os.path.join(folder_path, f"{row['ID']}_clear.gpx")
        if os.path.exists(clear_file):
            clear_times.append(extract_time(clear_file))
        else:
            clear_times.append(None)
    else:
        clear_times.append(None)

    # Alt Route total time
    if row["Alt_Route"] is not None:
        alt_file = os.path.join(folder_path, f"{row['ID']}_alt.gpx")
        if os.path.exists(alt_file):
            alt_times.append(extract_time(alt_file))
        else:
            alt_times.append(None)
    else:
        alt_times.append(None)

# Add the total times to the DataFrame
df["Clear_Time"] = clear_times
df["Alt_Time"] = alt_times

In [156]:
# Function to calculate slope in percentage for a route
def calculate_slope_in_percentage(route):
    for i, point in enumerate(route):
        if i == 0:  # First point, slope is 0
            point["slope"] = 0
        else:
            # Calculate elevation difference and distance
            elevation_diff = point["elevation"] - route[i - 1]["elevation"]
            # Distance between current and previous point
            distance = point["distance"]
            # Calculate slope as percentage (rise/run * 100), avoid division by zero
            point["slope"] = (elevation_diff / distance *
                              100) if distance != 0 else 0
    return route


# Apply the function to each row in the DataFrame
for index, row in df.iterrows():
    if "Clear_Route" in row and row["Clear_Route"]:
        df.at[index, "Clear_Route"] = calculate_slope_in_percentage(
            row["Clear_Route"])
    if "Alt_Route" in row and row["Alt_Route"]:
        df.at[index, "Alt_Route"] = calculate_slope_in_percentage(
            row["Alt_Route"])

In [157]:
row_index = 34
df.iloc[row_index]["Clear_Route"]

[{'point_index': 0,
  'longitude': 17.677152,
  'latitude': 48.136723,
  'distance': 0,
  'cumulative_distance': 0,
  'elevation': 113.4,
  'slope': 0},
 {'point_index': 1,
  'longitude': 17.67615,
  'latitude': 48.139437,
  'distance': 311.155304280269,
  'cumulative_distance': 311.155304280269,
  'elevation': 115.1,
  'slope': 0.5463509625626489},
 {'point_index': 2,
  'longitude': 17.676018,
  'latitude': 48.139747,
  'distance': 35.87512936353681,
  'cumulative_distance': 347.0304336438058,
  'elevation': 115.3,
  'slope': 0.557489278918897},
 {'point_index': 3,
  'longitude': 17.675871,
  'latitude': 48.14002,
  'distance': 32.29255765316188,
  'cumulative_distance': 379.32299129696764,
  'elevation': 115.5,
  'slope': 0.619337750041673},
 {'point_index': 4,
  'longitude': 17.67549,
  'latitude': 48.140602,
  'distance': 70.7000239549139,
  'cumulative_distance': 450.02301525188153,
  'elevation': 115.9,
  'slope': 0.5657706711034349},
 {'point_index': 5,
  'longitude': 17.67547,


In [158]:
row_index = 34
df.iloc[row_index]["ID"]

'M1829'

In [159]:
def get_fuel_consumption(slope):
    return 6 + 0.766 * slope + 0.0646 * slope**2


def calculate_total_consumption(route):
    total_fuel_consumption = 0
    for point in route:
        slope = point["slope"]
        # Convert distance to kilometers
        distance_km = point["distance"] / 1000
        distance_per_100km = distance_km / 100  # Convert distance to 100 km units
        # Get instantaneous fuel consumption
        fuel_per_100km = get_fuel_consumption(slope)
        segment_fuel = fuel_per_100km * distance_per_100km  # Fuel for this segment
        total_fuel_consumption += segment_fuel
    return total_fuel_consumption


# Apply the corrected function to calculate total consumption for each route
for index, row in df.iterrows():
    if "Clear_Route" in row and row["Clear_Route"]:
        df.at[index, "Clear_Consumption"] = calculate_total_consumption(
            row["Clear_Route"])
    if "Alt_Route" in row and row["Alt_Route"]:
        df.at[index, "Alt_Consumption"] = calculate_total_consumption(
            row["Alt_Route"])

In [160]:
# Calculate differences
df['Distance_Difference'] = df['Alt_Distance'] - df['Clear_Distance']
df['Time_Difference'] = df['Alt_Time'] - df['Clear_Time']
df['Consumption_Difference'] = df['Alt_Consumption'] - df['Clear_Consumption']

In [161]:
df['Time'] = (df['Time_Difference'] * df['Intenz'] / 60).astype(int)
df['Distance'] = (df['Distance_Difference'] * df['Intenz']).astype(int)
df['Price'] = (df['Distance_Difference'] * df['Intenz'] * 0.31).astype(int)
df['Env_Impact'] = df['Consumption_Difference'] * df['Intenz'] * 2.392E-3

In [162]:
df.columns

Index(['ID', 'GPS', 'Dlzka', 'PK', 'Intenz', 'Clear_Route', 'Alt_Route',
       'Clear_Distance', 'Alt_Distance', 'Clear_Time', 'Alt_Time',
       'Clear_Consumption', 'Alt_Consumption', 'Distance_Difference',
       'Time_Difference', 'Consumption_Difference', 'Time', 'Distance',
       'Price', 'Env_Impact'],
      dtype='object')

In [163]:
df.iloc[35]

ID                                                                    M1075
GPS                                              48.256613532, 17.764858116
Dlzka                                                                 103.6
PK                                                                     1345
Intenz                                                                 2116
Clear_Route               [{'point_index': 0, 'longitude': 17.78389, 'la...
Alt_Route                 [{'point_index': 0, 'longitude': 17.78389, 'la...
Clear_Distance                                                     5.894471
Alt_Distance                                                      18.633293
Clear_Time                                                        10.504117
Alt_Time                                                          20.068767
Clear_Consumption                                                  0.359976
Alt_Consumption                                                    1.125482
Distance_Dif

In [172]:
print(df[["ID", "Intenz", "Time", "Distance", "Price", "Env_Impact"]])

       ID  Intenz  Time  Distance   Price  Env_Impact
0   M6425   12447  2981    245291   76040   34.608106
1   M6684    3931  1375     86260   26740   15.696954
2   M4553   14134  7581    354543  109908   50.479152
3    M679    6052  1822    145635   45146   23.285599
4   M4864    3663   574     35934   11139    5.232961
5   M3839    7810   433      5403    1675    0.783504
6   M5081     882    37      2514     779    0.890354
7   M5770    6843   548     35215   10916    6.104616
8   M5237     882    37      2514     779    0.888306
9    M299    1310   205     11297    3502    1.895949
10  M6245   11674   279     17230    5341    2.438748
11  M2332   10818  1170     34599   10725    4.993954
12  M6224    5702   938     57400   17794    8.220549
13  M6252   11604   320     25717    7972    3.750016
14  M7235    5702  1059     59584   18471    8.523404
15   M283    6748   161      9959    3087    1.409753
16  M2276    2810   297     23438    7265    3.366803
17  M7011   11790   937     

In [165]:
import numpy as np
import plotly.graph_objects as go

# Define the slope range
slope_range = np.linspace(-5, 5, 100)

# Calculate fuel consumption using the provided function
fuel_consumption = [get_fuel_consumption(slope) for slope in slope_range]

# Create a Plotly figure
fig = go.Figure()

# Add the fuel consumption line
fig.add_trace(go.Scatter(
    x=slope_range,
    y=fuel_consumption,
    mode='lines',
    line=dict(color='blue', width=2),
    name='Fuel Consumption'
))

# Update the layout
fig.update_layout(
    title="Fuel Consumption vs. Slope",
    xaxis_title="Slope [%]",
    yaxis_title="Fuel Consumption [L/100km]",
    template="plotly_white",
    font=dict(family="Montserrat, sans-serif", size=12),
    width=800,
    height=500
)

# Show the plot
fig.show()

In [166]:
import plotly.graph_objects as go


def plot_elevation_profile(row, verbose=False, showfig=True, savefig=False):
    # Extract the ID for naming
    ajDi = df.iloc[row]["ID"]

    # Extract Clear and Alt routes for the specified row
    clear_route = df.iloc[row]["Clear_Route"]
    alt_route = df.iloc[row]["Alt_Route"]

    if verbose:
        print(f"Processing row {row} with ID {ajDi}...")
        print(f"Clear Route Points: {
              len(clear_route)}, Alt Route Points: {len(alt_route)}")

    # Extract distances, elevations, and slopes for Clear and Alt routes
    clear_distances = [point["cumulative_distance"] /
                       1000 for point in clear_route]  # Convert to kilometers
    clear_elevations = [point["elevation"] for point in clear_route]
    clear_slopes = [point["slope"] for point in clear_route]

    alt_distances = [point["cumulative_distance"] /
                     1000 for point in alt_route]  # Convert to kilometers
    alt_elevations = [point["elevation"] for point in alt_route]
    alt_slopes = [point["slope"] for point in alt_route]

    # Calculate the elevation range
    min_elevation = min(min(clear_elevations), min(alt_elevations))
    max_elevation = max(max(clear_elevations), max(alt_elevations))

    # Add a buffer of ±5 meters
    y_min = min_elevation - 5
    y_max = max_elevation + 5

    # Create a Plotly figure
    fig = go.Figure()

    # Set font to Montserrat for the entire figure
    fig.update_layout(
        font=dict(
            family="Montserrat, sans-serif",  # Set the font family
            size=12  # Optional: Adjust the font size
        )
    )

    # Add the alternative route's elevation profile
    fig.add_trace(go.Scatter(
        x=alt_distances,
        y=alt_elevations,
        mode="lines",
        name="Obchádzka",
        line=dict(color="red"),
        fill='tozeroy',  # Fill area under the line to the x-axis (y=0)
        fillcolor='rgba(255, 99, 71, 0.3)'  # Transparent red fill
    ))

    # Add the clear route's elevation profile
    fig.add_trace(go.Scatter(
        x=clear_distances,
        y=clear_elevations,
        mode="lines",
        name="Priama trasa",
        line=dict(color="green"),
        fill='tozeroy',  # Fill area under the line to the x-axis (y=0)
        fillcolor='rgba(8, 132, 9, 0.3)'  # Transparent blue fill
    ))

    # Add the alternative route's slope as step-like lines
    fig.add_trace(go.Scatter(
        x=alt_distances,
        y=alt_slopes,
        mode="lines",
        name="Sklon obchádzky",
        line=dict(color="red", dash="solid", width=1),  # Set line width to 1
        opacity=0.7,  # Set line opacity to 0.7
        yaxis="y2",  # Use secondary Y-axis
        line_shape="hv"  # Step-like lines
    ))

    # Add the clear route's slope as step-like lines
    fig.add_trace(go.Scatter(
        x=clear_distances,
        y=clear_slopes,
        mode="lines",
        name="Sklon priamej trasy",
        line=dict(color="green", dash="solid", width=1),  # Set line width to 1
        opacity=0.7,  # Set line opacity to 0.7
        yaxis="y2",  # Use secondary Y-axis
        line_shape="hv"  # Step-like lines
    ))

    # Update the layout
    fig.update_layout(
        title=f"Výškové profily priamej trasy a obchádzky mosta {ajDi}",
        xaxis_title="Vzdialenosť [km]",
        yaxis=dict(
            title="Nadmorská výška [m n. m.]",
            range=[y_min, y_max],
            showgrid=True
        ),
        yaxis2=dict(
            title="Sklon [%]",
            overlaying="y",
            side="right",
            showgrid=False
        ),
        legend=dict(
            title="Legenda",
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        ),
        template="plotly_white",
        width=800,  # Set the width of the plot
        height=400   # Set the height of the plot
    )

    # Show the plot if requested
    if showfig:
        fig.show()

    # Save the plot to a file if requested
    if savefig:
        filename = f"{ajDi}_elev_profile.png"
        fig.write_image(filename)
        if verbose:
            print(f"Figure saved as {filename}")

In [167]:
# Call the function for row 18
plot_elevation_profile(row=35, verbose=False, showfig=True, savefig=True)

In [168]:
import plotly.graph_objects as go

# Specify the row index
row_index = 34

# Extract Clear and Alt routes for the specified row
clear_route = df.iloc[row_index]["Clear_Route"]
alt_route = df.iloc[row_index]["Alt_Route"]

# Extract distances and elevations for Clear and Alt routes
clear_distances = [point["cumulative_distance"] /
                   1000 for point in clear_route]  # Convert to kilometers
clear_elevations = [point["elevation"] for point in clear_route]

alt_distances = [point["cumulative_distance"] /
                 1000 for point in alt_route]  # Convert to kilometers
alt_elevations = [point["elevation"] for point in alt_route]

# Calculate the elevation range
min_elevation = min(min(clear_elevations), min(alt_elevations))
max_elevation = max(max(clear_elevations), max(alt_elevations))

# Add a buffer of ±5 meters
y_min = min_elevation - 5
y_max = max_elevation + 5

# Create a Plotly figure
fig = go.Figure()

# Add the alternative route's elevation profile
fig.add_trace(go.Scatter(
    x=alt_distances,
    y=alt_elevations,
    mode="lines",
    name="Alternative Route",
    line=dict(color="red"),
    fill='tozeroy',  # Fill area under the line to the x-axis (y=0)
    fillcolor='rgba(255, 99, 71, 0.3)'  # Transparent red fill
))

# Add the clear route's elevation profile
fig.add_trace(go.Scatter(
    x=clear_distances,
    y=clear_elevations,
    mode="lines",
    name="Clear Route",
    line=dict(color="blue"),
    fill='tozeroy',  # Fill area under the line to the x-axis (y=0)
    fillcolor='rgba(135, 206, 250, 0.3)'  # Transparent blue fill
))

# Update the layout
fig.update_layout(
    title=f"Elevation Profiles for Clear and Alternative Routes (Row {
        row_index})",
    xaxis_title="Distance (km)",
    yaxis_title="Elevation (m)",
    yaxis=dict(range=[y_min, y_max]),  # Set y-axis limits
    legend=dict(
        title="Route Type",
        orientation="h",  # Horizontal legend
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5
    ),
    template="plotly_white"
)

# Show the plot
fig.show()

In [169]:
import folium

# Specify the index to plot
index_to_plot = 35

# Extract Clear and Alt routes for the specified index
clear_route = df.iloc[index_to_plot]["Clear_Route"]
alt_route = df.iloc[index_to_plot]["Alt_Route"]

# Create a Folium map centered at the first point of the Clear Route (if available)
if clear_route:
    start_point = [clear_route[0]["latitude"], clear_route[0]["longitude"]]
elif alt_route:
    start_point = [alt_route[0]["latitude"], alt_route[0]["longitude"]]
else:
    start_point = [0, 0]  # Fallback if no routes are available

map_folium = folium.Map(location=start_point, zoom_start=14)

# Add Clear Route to the map
if clear_route:
    clear_coords = [(point["latitude"], point["longitude"])
                    for point in clear_route]
    folium.PolyLine(clear_coords, color="blue", weight=5,
                    opacity=0.7, tooltip="Clear Route").add_to(map_folium)

# Add Alt Route to the map
if alt_route:
    alt_coords = [(point["latitude"], point["longitude"])
                  for point in alt_route]
    folium.PolyLine(alt_coords, color="red", weight=5,
                    opacity=0.7, tooltip="Alt Route").add_to(map_folium)

# Display the map
map_folium

In [170]:
df.iloc[0]["Clear_Route"]

[{'point_index': 0,
  'longitude': 17.781844,
  'latitude': 48.589306,
  'distance': 0,
  'cumulative_distance': 0,
  'elevation': 157.0,
  'slope': 0},
 {'point_index': 1,
  'longitude': 17.782481,
  'latitude': 48.589233,
  'distance': 47.602716288981114,
  'cumulative_distance': 47.602716288981114,
  'elevation': 157.1,
  'slope': 0.21007204587428535},
 {'point_index': 2,
  'longitude': 17.782774,
  'latitude': 48.589219,
  'distance': 21.630571666253434,
  'cumulative_distance': 69.23328795523454,
  'elevation': 156.6,
  'slope': -2.311543160831327},
 {'point_index': 3,
  'longitude': 17.782993,
  'latitude': 48.589215,
  'distance': 16.131689660348066,
  'cumulative_distance': 85.3649776155826,
  'elevation': 156.4,
  'slope': -1.2397957325672564},
 {'point_index': 4,
  'longitude': 17.783217,
  'latitude': 48.589223,
  'distance': 16.517728230972956,
  'cumulative_distance': 101.88270584655555,
  'elevation': 156.3,
  'slope': -0.6054101302652558},
 {'point_index': 5,
  'longitud